# Budget Formulation


In this program we will demonstrate how to generate simulated total to landowners in order to formulate an accurate budget and allocate resources for an upcoming commercial project. 

Please note that all data is simulated for the purposes of this project. 

# Scenario

In this scenario we will be constructing new oil pipeline infrastructure in West Texas in which we need to estimate how much compensation we can expect to pay landowners to allow us to construct on their lands. 

Therefore, we will perform a Monte Carlo Simulation Model based on prior assumptions to estimate the total compensation so that we may add this to the total pipeline budget. 


---


The simulation assumptions used are as follows:


*   Compensation demanded from Landowners are independent from each other and are paid by the linear feet of pipeline on their land.
*   There are two types of Landowners:
  * Extreme: Demands twice as much money than most landowners
  * Normal: The average landowner that demands identical amounts as its peers. 
* Landowner Type Probability Distribution
  * Landowners have a random uniform distribution 0%-20% probability (**$\theta$**)  of being 'Extreme'. 
  * The distribution per simulaton of the individual Landowner Type is Bernoulli with the randomized paramter of *$\theta$* as the probability of success or failure. (i.e. Bernoulli(**$\theta$** ))
* Compensation Distribution:
  * All Landowners are assumed to have a Normal Distribution of compensation per foot demanded.
  * Normal Landowner: Average: \\$20/foot, Standard Deviation: \\$5/foot.
    * Simulation Cutoff: \\$10/foot (we do not realistically expect to pay less than \\$10/foot to land owners) 
  * Extreme Landowner: Average: \\$40/foot, Standard Deviation: \\$10/foot.
     * Simulation Cutoff: \\$30/foot (we do not realistically expect to pay less than \\$30/foot to extreme land owners) 

# Code Blocks


## Import Packages

In [ ]:
#import packages
import random
import math
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import plotly as plty
import plotly.express as px

## Initial Simulation Criteria

In [ ]:
#Set Initial Simulation Criteria
seed=1              #set seed for reproducible random output
simulations=10000   #set amount of simulations

## Dataframe Setup

In [ ]:
#Create Fake Landowner Dataset

lo={'Landowner_Index':[1,2,3,4,5,6,7,8,9,10]}
landowners= pd.DataFrame(lo)                 #Creates list of 10 Landowners
np.random.seed(seed)
Linear_Feet=pd.DataFrame()
landowners['Linear_Feet']=0
for i in range(0,10):
  landowners['Linear_Feet'][i]=np.random.uniform(low=1000, high=20000) #Get Random Linear Feet for each landowner from 1000-20000

landowners

,Landowner_Index,Linear_Feet
0,1,8923
1,2,14686
2,3,1002
3,4,6744
4,5,3788
5,6,2754
6,7,4538
7,8,7565
8,9,8538
9,10,11237


In [ ]:
landowner_type=pd.DataFrame()
landowners['Type']=0
landowners['Payout/Foot']=0
landowners['Total Payout']=0
landowners

,Landowner_Index,Linear_Feet,Type,Payout/Foot,Total Payout
0,1,8923,0,0,0
1,2,14686,0,0,0
2,3,1002,0,0,0
3,4,6744,0,0,0
4,5,3788,0,0,0
5,6,2754,0,0,0
6,7,4538,0,0,0
7,8,7565,0,0,0
8,9,8538,0,0,0
9,10,11237,0,0,0


## Simulation Assumptions

In [ ]:
lower_bound=0   #lower bound probability of extreme landowner
upper_bound=.20 #upper bound proabability of extreme landowner

mean_normal=20  #mean compensation for normal landowner
std_normal=5    #standard deviationi of compensation for normal landowner
mean_extreme=40 #mean compensation for extreme landowner
std_extreme=10  #standard deviationi of compensation for extreme landowner

## Model

In [ ]:
np.random.seed(seed)
list1=range(0,simulations)
landowners_total_payout=pd.DataFrame(list1)
landowners_total_payout['Total_Compensation']=0


for j in range(0,simulations):
  for i in range(0,len(landowners)):
    p=np.random.uniform(low=lower_bound,high=upper_bound)
    landowners['Type'][i]=np.random.binomial(n=1,p=p)
    if landowners['Type'][i]==0:
      landowners['Payout/Foot'][i]=np.random.normal(loc=mean_normal,scale=std_normal)
      if landowners['Payout/Foot'][i] <10:
        landowners['Payout/Foot'][i]=10
      else: pass
    if landowners['Type'][i]==1:
      landowners['Payout/Foot'][i]=np.random.normal(loc=mean_extreme,scale=std_extreme)
      if landowners['Payout/Foot'][i] <30:
        landowners['Payout/Foot'][i]=30
      else: pass
    landowners['Total Payout'][i]=landowners['Linear_Feet'][i]*landowners['Payout/Foot'][i]
  landowners_total_payout['Total_Compensation'][j]=sum(landowners['Total Payout'])


# Output & Conclusion


Below is the output for the assumptions used and the data given based on 10,000 iterations.

As can be seen the expected (mean) payout will be \$1,509,603 with a 95% Confidence Bound of $1,883,966 and a 99% Confidence Bound of \$2,089,535. 

Therefore, in this scenario we can expect to pay landowners somewhere near ~$1.5M and should create the budget based on the 95% or 99% Confidence bound (depending on the acceptable level of risk) to allow for overages. 

In [ ]:
quantiles=landowners_total_payout['Total_Compensation'].quantile([.5,.75,.9,.95,.99])
mean=landowners_total_payout['Total_Compensation'].mean()

print('Simulated Quantiles:\n',quantiles,"\n Mean:\n",mean)

Simulated Quantiles:
 0.50    1482807.50
0.75    1631142.25
0.90    1787398.10
0.95    1883966.55
0.99    2089535.75
Name: Total_Compensation, dtype: float64 
 Mean:
 1509603.1405


In [ ]:
sim_histogram=px.histogram(landowners_total_payout,x='Total_Compensation', title='Simulated Total Landowner Compensation')
sim_histogram